# Digital Twin Fairways
WIP to use the OpenTNSim engine

In [1]:
import datetime
import json
import logging
import time

import dtv_backend.berthing

# the simpy processes and objects
import dtv_backend.compat

# library to load the fairway information network
import dtv_backend.fis
import dtv_backend.network
import dtv_backend.network.network_utilities
import dtv_backend.postprocessing
import dtv_backend.simple
import dtv_backend.simulate
import geojson
import networkx as nx
import pandas as pd
import shapely
import simpy
from networkx.readwrite import json_graph

# opentnsim import
from opentnsim import core

# reload for debugging purposes
%load_ext autoreload
%autoreload 2

/Users/baart_f/.virtualenvs/main/lib/python3.9/site-packages/geopandas/_compat.py:111: UserWarning:

The Shapely GEOS version (3.10.2-CAPI-1.16.0) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.



### Input
You can define your input in a json configuration file. The relevant parts are sites, fleet and climate.

In [2]:
# example input
with open("config.geojson") as f:
    config = geojson.load(f)

### Fixed content

In [3]:
Port = type(
    "Port",
    (
        dtv_backend.compat.Processor,
        core.Identifiable,
        core.Locatable,
        core.HasResource,
        core.HasContainer,
        core.ExtraMetadata,
    ),
    {},
)

In [4]:
Ship = type(
    "Ship",
    (
        dtv_backend.compat.CanWork,
        dtv_backend.berthing.CanBerth,
        core.Identifiable,
        core.HasContainer,
        core.Movable,
        core.Locatable,
        core.ExtraMetadata,
    ),
    {},
)

### What is to be v3_run

In [5]:
# always start at now
now = datetime.datetime.now()
initial_time = now.timestamp()
env = simpy.Environment(initial_time=initial_time)
env.epoch = now

# default no berth
with_berth = config.get("options", {}).get("with_berth", False)

# read the network from google for performance reasons
url = "https://zenodo.org/record/6673604/files/network_digital_twin_v0.3.pickle?download=1"
G = dtv_backend.fis.load_fis_network(url)
env.FG = G

/Users/baart_f/src/digitaltwin-waterway/dtv_backend/dtv_backend/fis.py:112: ShapelyDeprecationWarning:

The array interface is deprecated and will no longer work in Shapely 2.0. Convert the '.coords' to a numpy array instead.



In [7]:
# ports
# logger.info("Loading ports ⚓")
ports = []
for site in config["sites"]:
    # port = dtv_backend.simple.Port(env, **site["properties"], **site)
    node = site["properties"]["n"]
    loading_rate = site["properties"]["loadingRate"]
    loading_rate_variation = site["properties"]["loadingRateVariation"]
    port = Port(
        env=env,
        node=node,
        loading_rate=loading_rate,
        loading_rate_variation=loading_rate_variation,
        **site["properties"],
        **site
    )
    ports.append(port)

In [8]:
# ships
# logger.info("Loading ships 🚢")
ships = []
for ship in config["fleet"]:
    kwargs = {}
    kwargs.update(ship)
    kwargs.update(ship["properties"])
    kwargs["v"] = 3
    kwargs["route"] = [feature["properties"]["n"] for feature in config["route"]]
    geometry = shapely.geometry.shape(ship["geometry"])
    node, dist = dtv_backend.fis.find_closest_node(env.FG, geometry)
    kwargs["node"] = node
    # the ship needs to know about the climate
    if "climate" in config:
        kwargs["climate"] = config["climate"]
    # ship = dtv_backend.simple.Ship(env, **kwargs)
    ship = Ship(env=env, **kwargs)
    ships.append(ship)

In [9]:
route = [feature["properties"]["n"] for feature in config["route"]]

In [10]:
# vars(ship)

In [11]:
# operator
# logger.info("Loadig operator 👩‍💼")
# Setup and start the simulation
operator = dtv_backend.simple.Operator(env=env, ships=ships, **config["operator"])
# The ships do work for the operator
for ship in ships:
    env.process(ship.work_for(operator, with_berth=with_berth))
# The opertor plans the work move everything from A to B
env.process(operator.plan(ports[0], ports[1]))

<Process(plan) object at 0x2bb409970>

In [12]:
# logger.info("Running simulation 👩‍💻")
# Run for n days
n_days_in_future = now + datetime.timedelta(days=60)
env.run(until=n_days_in_future.timestamp())

### WIP snippets

In [13]:
pd.DataFrame(operator.logbook)

,Message,Timestamp,Value,geometry,ActivityID,ActivityState,Meta
0,Plan,2022-07-13 12:04:50.260389,NaN,None,1,None,"{'state': 'START', 'actor': <dtv_backend.simpl..."
1,Task,2022-07-13 12:04:50.260389,NaN,None,2,None,"{'state': 'START', 'actor': <dtv_backend.simpl..."
2,Task,2022-07-13 13:04:50.260389,NaN,None,2,None,"{'state': 'STOP', 'actor': <dtv_backend.simple..."
3,Task,2022-07-13 13:04:50.260389,NaN,None,3,None,"{'state': 'START', 'actor': <dtv_backend.simpl..."
4,Cycle,2022-07-13 13:04:50.260389,NaN,None,4,None,"{'state': 'START', 'actor': <__main__.Ship obj..."
...,...,...,...,...,...,...,...
59,Sailing,2022-07-15 16:59:09.628819,107963.754982,"LINESTRING (4.09001864901624 51.9721147516422,...",32,None,"{'state': 'START', 'actor': <__main__.Ship obj..."
60,Sailing,2022-07-16 02:58:57.547146,107963.754982,"LINESTRING (4.09001864901624 51.9721147516422,...",32,None,"{'state': 'STOP', 'actor': <__main__.Ship obje..."
61,Unload request,2022-07-16 02:58:57.547146,NaN,POINT (4.934845473565718 52.03883776809355),33,None,"{'state': 'START', 'actor': <__main__.Ship obj..."
62,Unload request,2022-07-16 02:58:57.547146,NaN,POINT (4.934845473565718 52.03883776809355),33,None,"{'state': 'STOP', 'actor': <__main__.Ship obje..."


In [15]:
ship.logbook is operator.logbook

True